In [ ]:
%load_ext autoreload
%autoreload 2


# from backend.database.all_models import get_all_models
import pandas as pd
# get_all_models()
from backend.models import Classroom, Student, Exam, ExamResult,Module
from backend.database.database import get_db
import uuid


In [ ]:

db = get_db()

def get_overall_results(classroom_id: uuid.UUID):
    results = (
        db.query(
            Classroom.name.label('classroom_name'),
            Student.name.label('student_name'),
            Module.name.label('module_name'),
            Exam.name.label('exam_name'),
            ExamResult.marks_obtained,
            Exam.total_marks,
            (ExamResult.marks_obtained / Exam.total_marks * 100).label('percentage')
        )
        .join(Student, Student.classroom_id == Classroom.id)
        .join(ExamResult, ExamResult.student_id == Student.id)
        .join(Exam, Exam.id == ExamResult.exam_id)
        .join(Module, Module.id == Exam.module_id)
        .filter(Classroom.id == classroom_id)
        .all()
    )
    
    return results


# Example usage
results = get_overall_results(uuid.UUID("96f5ee77-9bf3-47e4-a936-b15d741b53eb"))

# Convert results to a pandas DataFrame
df = pd.DataFrame(results, columns=['classroom_name', 'student_name', 'module_name', 'exam_name', 'marks_obtained', 'total_marks', 'percentage'])

# Pivot the DataFrame to get the desired format
df_pivot = df.pivot_table(index='student_name', columns='module_name', values='marks_obtained', aggfunc='sum').fillna(0)

# Calculate total marks and percentage for each student
df_pivot['TOTAL_MARKS'] = df_pivot.sum(axis=1)
df_pivot['PERCENTAGE'] = (df_pivot['TOTAL_MARKS'] / (df['total_marks'].max() * len(df_pivot.columns))) * 100
df_pivot['MEAN'] = df_pivot.mean(axis=1)

# Reset the index to make 'student_name' a column
df_pivot.reset_index(inplace=True)

# Sort by percentage in descending order
df_sorted = df_pivot.sort_values(by='PERCENTAGE', ascending=False)

print(df_sorted)